In [2]:
import pandas as pd
import numpy as np
import json
import matplotlib.pyplot as plt

transaction_data = pd.read_csv('transactions_train.csv')
users_data = pd.read_csv('users_data.csv')
cards_data = pd.read_csv('cards_data.csv')
with open('mcc_codes.json', 'r') as file:
    mcc_codes = json.load(file)
with open('train_fraud_labels.json', 'r') as file:
    train_fraud_label = json.load(file)
train_fraud_label = train_fraud_label['target']
final_df = pd.read_csv('final_df.csv')

In [10]:
targets = pd.DataFrame(list(train_fraud_label.items()), columns=['transaction_id', 'target'])
targets['target'] = targets['target'].replace({'No': 0, 'Yes': 1})
#targets.head()
transaction_data['transaction_id'] = transaction_data['transaction_id'].astype(str)
merged_df = transaction_data.merge(targets, left_on='transaction_id', right_on='transaction_id', how='left')
merged_df.head()

,transaction_id,date,client_id,card_id,amount,use_chip,merchant_id,merchant_city,merchant_state,zip,mcc,errors,target
0,19765990,2017-07-07 09:53:00,1581,2519,$12.35,Chip Transaction,20519,Arnold,MO,63010.0,5942,NaN,0
1,22160255,2018-11-26 17:44:00,1862,4049,$58.29,Chip Transaction,98648,Des Moines,IA,50313.0,5814,NaN,0
2,17566794,2016-03-26 12:42:00,1967,3367,$11.03,Chip Transaction,46978,Lake Forest,CA,92630.0,5411,NaN,0
3,17318690,2016-02-01 08:30:00,921,3457,$85.74,Chip Transaction,63701,Rush,NY,14543.0,5411,NaN,0
4,20994060,2018-03-24 14:42:00,456,2800,$13.43,Chip Transaction,83271,Estero,FL,33928.0,4214,NaN,0


In [14]:
print(merged_df.shape)
idx_df = merged_df['client_id'].unique()
idx_df.shape
cards = cards_data[cards_data['client_id'].isin(idx_df)]
idx_cards = cards['client_id'].unique()
users = users_data[users_data['id'].isin(idx_cards)]
users.head()

(210000, 13)


,id,current_age,retirement_age,birth_year,birth_month,gender,address,latitude,longitude,per_capita_income,yearly_income,total_debt,credit_score,num_credit_cards
1,1746,53,68,1966,12,Female,3606 Federal Boulevard,40.76,-73.74,$37891,$77254,$191349,701,5
6,1075,36,67,1983,12,Female,5695 Fifth Street,38.22,-85.74,$25258,$51500,$102286,672,3
13,1094,34,62,1985,10,Male,74786 Jefferson Drive,44.75,-85.60,$20325,$41442,$78833,712,3
15,1660,41,68,1978,4,Female,40 Washington Drive,36.73,-102.51,$11342,$23123,$5079,723,6
19,511,66,68,1953,10,Male,153 Tenth Lane,32.21,-110.88,$17460,$35602,$55369,661,5


In [16]:
users = users.rename(columns={'id': 'client_id'})
users.head()
users.drop(columns=['address'], inplace=True)
users_cards = users.merge(cards, on='client_id', how='inner')
users_cards.head()

,client_id,current_age,retirement_age,birth_year,birth_month,gender,latitude,longitude,per_capita_income,yearly_income,...,card_type,card_number,expires,cvv,has_chip,num_cards_issued,credit_limit,acct_open_date,year_pin_last_changed,card_on_dark_web
0,1746,53,68,1966,12,Female,40.76,-73.74,$37891,$77254,...,Credit,4404898874682993,09/2003,736,YES,1,$27500,09/2003,2012,No
1,1746,53,68,1966,12,Female,40.76,-73.74,$37891,$77254,...,Debit,4001482973848631,07/2022,972,YES,2,$28508,02/2011,2011,No
2,1746,53,68,1966,12,Female,40.76,-73.74,$37891,$77254,...,Debit,5627220683410948,06/2022,48,YES,2,$9022,07/2003,2015,No
3,1746,53,68,1966,12,Female,40.76,-73.74,$37891,$77254,...,Debit (Prepaid),5711382187309326,11/2020,722,YES,2,$54,06/2010,2015,No
4,1746,53,68,1966,12,Female,40.76,-73.74,$37891,$77254,...,Debit (Prepaid),5766121508358701,02/2023,908,YES,1,$99,07/2006,2012,No


In [21]:
final_df = merged_df.merge(users_cards, on='client_id', how='inner')
target_column = final_df.pop('target')
final_df['target'] = target_column
final_df.head()
print(final_df.columns)

Index(['transaction_id', 'date', 'client_id', 'card_id', 'amount', 'use_chip',
       'merchant_id', 'merchant_city', 'merchant_state', 'zip', 'mcc',
       'errors', 'current_age', 'retirement_age', 'birth_year', 'birth_month',
       'gender', 'latitude', 'longitude', 'per_capita_income', 'yearly_income',
       'total_debt', 'credit_score', 'num_credit_cards', 'id', 'card_brand',
       'card_type', 'card_number', 'expires', 'cvv', 'has_chip',
       'num_cards_issued', 'credit_limit', 'acct_open_date',
       'year_pin_last_changed', 'card_on_dark_web', 'target'],
      dtype='object')


In [25]:
final_df.drop(columns=['transaction_id', 'client_id', 'date', 'card_id', 'mcc', 'merchant_id', 'merchant_state', 'zip', 'mcc', 'errors', 'birth_year', 'birth_month', 'id'], inplace=True)

In [26]:
print(final_df.columns)

Index(['amount', 'use_chip', 'merchant_city', 'current_age', 'retirement_age',
       'gender', 'latitude', 'longitude', 'per_capita_income', 'yearly_income',
       'total_debt', 'credit_score', 'num_credit_cards', 'card_brand',
       'card_type', 'card_number', 'expires', 'cvv', 'has_chip',
       'num_cards_issued', 'credit_limit', 'acct_open_date',
       'year_pin_last_changed', 'card_on_dark_web', 'target'],
      dtype='object')


In [ ]:
final_df.drop(columns = ['cvv','has_chip','card_on_dark_web'],inplace=True)
print(final_df.columns)

In [ ]:
final_df.drop(columns = ['acct_open_date'], inplace=True)
print(final_df.head())

In [37]:
final_df.drop(columns = ['card_number'], inplace=True)
print(final_df.head())

   amount          use_chip merchant_city  current_age  retirement_age  \
0  $12.35  Chip Transaction        Arnold           38              65   
1  $12.35  Chip Transaction        Arnold           38              65   
2  $12.35  Chip Transaction        Arnold           38              65   
3  $12.35  Chip Transaction        Arnold           38              65   
4  $12.35  Chip Transaction        Arnold           38              65   

   gender  latitude  longitude per_capita_income yearly_income total_debt  \
0  Female     38.42     -90.36            $21744        $44334     $55173   
1  Female     38.42     -90.36            $21744        $44334     $55173   
2  Female     38.42     -90.36            $21744        $44334     $55173   
3  Female     38.42     -90.36            $21744        $44334     $55173   
4  Female     38.42     -90.36            $21744        $44334     $55173   

   credit_score  num_credit_cards  card_brand card_type  expires  \
0           684         

In [ ]:
final_df.drop(columns = ['expires'], inplace=True)
print(final_df.head())

   amount          use_chip merchant_city  current_age  retirement_age  \
0  $12.35  Chip Transaction        Arnold           38              65   
1  $12.35  Chip Transaction        Arnold           38              65   
2  $12.35  Chip Transaction        Arnold           38              65   
3  $12.35  Chip Transaction        Arnold           38              65   
4  $12.35  Chip Transaction        Arnold           38              65   

   gender  latitude  longitude per_capita_income yearly_income total_debt  \
0  Female     38.42     -90.36            $21744        $44334     $55173   
1  Female     38.42     -90.36            $21744        $44334     $55173   
2  Female     38.42     -90.36            $21744        $44334     $55173   
3  Female     38.42     -90.36            $21744        $44334     $55173   
4  Female     38.42     -90.36            $21744        $44334     $55173   

   credit_score  num_credit_cards  card_brand card_type  num_cards_issued  \
0           684

In [45]:
final_df = final_df[10000:]

In [46]:
monetary_columns = ['amount', 'per_capita_income', 'yearly_income', 'total_debt', 'credit_limit']
for col in monetary_columns:
    final_df[col] = final_df[col].replace('[\$,]', '', regex=True).astype(float)
categorical_cols = ['use_chip', 'merchant_city', 'gender', 'card_brand', 'card_type']
numerical_cols = final_df.drop(columns=categorical_cols + ['target']).select_dtypes(include=['int64', 'float64']).columns.tolist()
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer

numerical_pipeline = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

categorical_pipeline = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer(transformers=[
    ('num', numerical_pipeline, numerical_cols),
    ('cat', categorical_pipeline, categorical_cols)
])



<ipython-input-46-08d6ca236f39>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df[col] = final_df[col].replace('[\$,]', '', regex=True).astype(float)
<ipython-input-46-08d6ca236f39>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df[col] = final_df[col].replace('[\$,]', '', regex=True).astype(float)
<ipython-input-46-08d6ca236f39>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the cave

In [48]:
X = final_df.drop('target', axis=1)
y = final_df['target']
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline

# Create a pipeline that combines preprocessing and the classifier
model_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', RandomForestClassifier(random_state=42, max_depth=10, n_estimators=100, min_samples_split=2, min_samples_leaf=1, max_features='sqrt', class_weight='balanced', n_jobs=-1))
])

# Train the model
model_pipeline.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['amount', 'current_age',
                                                   'retirement_age', 'latitude',
                                                   'longitude',
                                                   'per_capita_income',
                                                   'yearly_income',
                                                   'total_debt', 'credit_score',
                                                   'num_credit_cards',
                                                   'num_cards_issued',
                                                   'credit_limit',
                                                   'year_pin_last_changed']),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('encoder',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['use_chip', 'merchant_city',
                                                   'gender', 'card_brand',
                                                   'card_type'])])),
                ('classifier',
                 RandomForestClassifier(class_weight='balanced', max_depth=10,
                                        n_jobs=-1, random_state=42))])

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

# Predict on the test set
y_pred = model_pipeline.predict(X_test)

# Evaluate the model
print("Confusion Matrix (Labels: 0 = Not Fraud, 1 = Fraud):")
print(confusion_matrix(y_test, y_pred, labels=[0, 1]))

print("\nClassification Report (Labels: 0 = Not Fraud, 1 = Fraud):")
print(classification_report(y_test, y_pred, labels=[0, 1], target_names=["Not Fraud (0)", "Fraud (1)"]))

print("\nAccuracy Score:")
print(accuracy_score(y_test, y_pred))



Confusion Matrix:
[[141417  17181]
 [    74    157]]

Classification Report:
              precision    recall  f1-score   support

           0       1.00      0.89      0.94    158598
           1       0.01      0.68      0.02       231

    accuracy                           0.89    158829
   macro avg       0.50      0.79      0.48    158829
weighted avg       1.00      0.89      0.94    158829


Accuracy Score:
0.891361149412261


In [50]:
from sklearn.model_selection import cross_val_score

# Perform 5-fold cross-validation
cv_scores = cross_val_score(model_pipeline, X, y, cv=5, scoring='accuracy')

# Output the cross-validation scores
print("Cross-validation scores:", cv_scores)
print("Mean accuracy:", cv_scores.mean())
print("Standard deviation:", cv_scores.std())


Cross-validation scores: [0.89707169 0.89479815 0.88949052 0.88136223 0.90630745]
Mean accuracy: 0.8938060076665406
Standard deviation: 0.008263246947980142
